# ClipCard Field Evaluation: Civic/ICS
**Purpose:** This notebook analyzes historical data from the `clipmap` CSVs to evaluate the real-world performance of the ClipCard system over time for the **Civic/ICS** silo.
It calculates and visualizes key metrics as defined in the `dfm-civic-ics.md` framework document, such as:- Near Miss Rate- Avoidable Reversals- Recheck On-Time %

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport matplotlib.dates as mdates# Load the aggregated ClipCard datatry:    cards_df = pd.read_csv('../../data/clipmap/cards.csv')    events_df = pd.read_csv('../../data/clipmap/events.csv')    # Filter for the correct domain and merge    ics_cards = cards_df[cards_df['domain'] == 'dfm-civic-ics'].copy()    merged_df = pd.merge(ics_cards, events_df, left_on='id', right_on='card_id')    merged_df['time'] = pd.to_datetime(merged_df['time'])except FileNotFoundError as e:    print(f'ERROR: {e.filename} not found. Please ensure the data exists.')    merged_df = pd.DataFrame()

### Metric 1: Near Miss Rate
**Definition:** Near-miss notes per Operational Period for high-risk interventions.**Goal:** Increase vs. baseline (better detection).
**Note:** This metric requires data from After-Action Reviews (AARs) or specific 'near miss' event types, which are not in the current `events.csv`. We will create a placeholder visualization with synthetic data.

In [ ]:
if not merged_df.empty:    # Group by week for trend analysis    events_by_week = merged_df.set_index('time').resample('W')        # --- Placeholder Data Generation ---     # In a real scenario, this would come from AAR logs or specific event types.    placeholder_df = events_by_week.size().to_frame(name='total_events')    # Assume a near-miss reporting rate that is increasing over time    placeholder_df['near_misses'] = np.random.randint(1, 5, size=len(placeholder_df)) + np.arange(len(placeholder_df))    placeholder_df['near_miss_rate'] = (placeholder_df['near_misses'] / placeholder_df['total_events']).fillna(0)    # --- End Placeholder ---     # Plotting    fig, ax = plt.subplots(figsize=(12, 6))    ax.plot(placeholder_df.index, placeholder_df['near_miss_rate'], marker='o', linestyle='-', label='Near Miss Rate (per OP)')    ax.set_title('Near Miss Reporting Rate Over Time (Synthetic Data)')    ax.set_ylabel('Rate (Near Misses per OP)')    ax.set_xlabel('Week')    ax.grid(True, which='both', linestyle='--', linewidth=0.5)    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))    plt.xticks(rotation=45)    ax.legend()    plt.tight_layout()    plt.show()else:    print('Data is empty. Cannot calculate Near Miss Rate.')

### Metric 2: Avoidable Reversals
**Definition:** Reversals/diversions judged avoidable by After-Action Review (AAR).**Goal:** Decrease vs. baseline.
**Note:** This also requires AAR data. We will use 'Incident Prevented' as a proxy for a 'reversal' and generate synthetic data for the 'avoidable' judgment.

In [ ]:
if not merged_df.empty:    reversals_df = merged_df[merged_df['outcome'] == 'Incident Prevented'].copy()    reversals_by_week = reversals_df.set_index('time').resample('W').size().to_frame(name='reversals')        # --- Placeholder Data Generation ---     # Assume the proportion of reversals judged 'avoidable' decreases over time as the system learns    avoidable_proportion = 0.5 - (np.arange(len(reversals_by_week)) * 0.1)    avoidable_proportion[avoidable_proportion < 0.1] = 0.1 # Floor at 10%    reversals_by_week['avoidable_reversals'] = (reversals_by_week['reversals'] * avoidable_proportion).astype(int)    reversals_by_week['avoidable_rate'] = (reversals_by_week['avoidable_reversals'] / reversals_by_week['reversals'] * 100).fillna(0)    # --- End Placeholder ---         # Plotting    fig, ax = plt.subplots(figsize=(12, 6))    ax.plot(reversals_by_week.index, reversals_by_week['avoidable_rate'], marker='s', linestyle='--', color='orange', label='Avoidable Reversal Rate (%)')    ax.set_title('Rate of Avoidable Reversals Over Time (Synthetic Data)')    ax.set_ylabel('Avoidable Rate (%)')    ax.set_xlabel('Week')    ax.grid(True, which='both', linestyle='--', linewidth=0.5)    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))    plt.xticks(rotation=45)    ax.legend()    plt.tight_layout()    plt.show()else:    print('Data is empty. Cannot calculate Avoidable Reversals.')